In [1]:
import os
import glob
import platform
from shutil import copy
from typing import List, Dict, Optional, Union, Tuple
from utils.command_utils import Command, DependencyError, ConversionError, TmpDir, LogFile, File, NiiFile

In [2]:
# Wrote some more text here

In [46]:
test_dir = "dir.eoiej"; os.path.abspath(test_dir)

'/Users/adebayobraimah/Desktop/projects/convert_source/bin/dir.eoiej'

In [27]:
with TmpDir(tmp_dir=os.getcwd(),use_cwd=True) as tmp_dir:
    tmp_dir.mk_tmp_dir()
    # tmp_dir.rm_tmp_dir(rm_parent=False)

In [28]:
os.listdir(os.getcwd())

['convert_source.py',
 '__init__.py',
 'utils',
 'run1.py',
 '__pycache__',
 'convert_source_dcm.py',
 'convert_source_par.py',
 'conversion_func_1.ipynb',
 '.ipynb_checkpoints',
 'tmp_dir_4541',
 'convert_source_nii.py']

In [5]:
def convert_image_data(file: str,
                       basename: str,
                       out_dir: str,
                       cprss_lvl: int = 6,
                       bids: bool = True,
                       anon_bids: bool = True,
                       gzip: bool = True,
                       comment: bool = True,
                       adjacent: bool = False,
                       dir_search: int = 5,
                       nrrd: bool = False,
                       ignore_2D: bool = True,
                       merge_2D: bool = True,
                       text: bool = False,
                       progress: bool = False,
                       verbose: bool = False,
                       write_conflicts: str = "suffix",
                       crop_3D: bool = False,
                       lossless: bool = False,
                       big_endian: str = "o",
                       xml: bool = False,
                       log: Optional[LogFile] = None,
                       env: Optional[Dict] = None,
                       dryrun: bool = False
                       ) -> BIDSimg:
                    #   ) -> Tuple[List[str],List[str],List[str],List[str]]:
    '''Converts medical image data (DICOM, PAR REC, or Bruker) to NifTi (or NRRD) using dcm2niix.
    This is a wrapper function for dcm2niix (v1.0.20190902+).

    Usage example:
        >>> data = convert_image_data("IM00001.dcm",
        ...                           "img0001",
        ...                           "<path/to/out/directory>")
        >>> data.imgs[0]
        "<path/to/image.nii>"
        >>>
        >>> data.jsons[0]
        "<path/to/image.json>"
        >>>
        >>> data.bvals[0]
        "<path/to/image.bval>"
        >>>
        >>> data.bvecs[0]
        "<path/to/image.bvec>"

    Arguments:
        file: Absolute path to raw image data file.
        basename: Output file(s) basename.
        out_dir: Absolute path to output directory (must exist at runtime).
        cprss_lvl: Compression level [1 - 9] - 1 is fastest, 9 is smallest (default: 6).
        bids: BIDS (JSON) sidecar (default: True).
        anon_bids: Anonymize BIDS (default: True).
        gzip: Gzip compress images (default: True).
        comment: Image comment(s) stored in NifTi header (default: True).
        adjacent: Assumes adjacent DICOMs/Image data (images from same series always in same folder) for faster conversion (default: False).
        dir_search (int): Directory search depth (default: 5).
        nrrd: Export as NRRD instead of NifTi, not recommended (default: False).
        ignore_2D: Ignore derived, localizer and 2D images (default: True).
        merge_2D: Merge 2D slices from same series regardless of echo, exposure, etc. (default: True).
        text: Text notes includes private patient details in separate text file (default: False).
        progress: Report progress, slicer format progress information (default: True).
        verbose: Enable verbosity (default: False).
        write_conflicts: Write behavior for name conflicts:
            - 'suffix' = Add suffix to name conflict (default)
            - 'overwrite' = Overwrite name conflict
            - 'skip' = Skip name conflict
        crop_3D: crop 3D acquisitions (default: False).
        lossless: Losslessly scale 16-bit integers to use dynamic range (default: True).
        big_endian: Byte order:
            - 'o' = optimal/native byte order (default)
            - 'n' = little endian
            - 'y' = big endian
        xml: Slicer format features (default: False).
        log: LogFile object for logging.
        env: Path environment dictionary.
        dryrun: Perform dryrun (creates the command, but does not execute it).
        
    Returns:
        BIDSimg data object that contains:
            - imgs: List of NIFTI image files
            - jsons: Corresponding list of JSON file(s)
            - bvals: Corresponding bval file(s)
            - bvecs: Corresponding bvec file(s)

        * Tuple of four lists of strings that correspond to: *
            - NIFTI image files
            - Corresponding JSON file(s)
            - Corresponding bval file(s)
            - Corresponding bvec file(s)
    
    Raises:
        ConversionError: Error that arises if no converted (NIFTI) images are created.
    '''
    
    # Get OS platform and construct command line args
    if platform.system().lower() == 'windows':
        convert: Command = Command("dcm2niix.exe")
    else:
        convert: Command = Command("dcm2niix")

    # Boolean True/False options arrays
    bool_opts: List[Union[str,bool]] = [bids, anon_bids, gzip, comment, adjacent, nrrd, ignore_2D, merge_2D, text, verbose, lossless, progress, xml]
    bool_vars: List[str] = ["-b", "-ba", "-z", "-c", "-a", "-e", "-i", "-m", "-t", "-v", "-l", "--progress", "--xml"]

    # Initial option(s)
    if cprss_lvl:
        convert.cmd_list.append(f"-{cprss_lvl}")

    # Keyword option(s)
    if write_conflicts.lower() == "suffix":
        convert.cmd_list.append("-w")
        convert.cmd_list.append("2")
    elif write_conflicts.lower() == "overwrite":
        convert.cmd_list.append("-w")
        convert.cmd_list.append("1")
    elif write_conflicts.lower() == "skip":
        convert.cmd_list.append("-w")
        convert.cmd_list.append("0")
    
    if big_endian.lower() == "o":
        convert.cmd_list.append("--big_endian")
        convert.cmd_list.append("o")
    elif big_endian.lower() == "n":
        convert.cmd_list.append("--big_endian")
        convert.cmd_list.append("n")
    elif big_endian.lower() == "y":
        convert.cmd_list.append("--big_endian")
        convert.cmd_list.append("y")
    
    for opt in zip(bool_opts,bool_vars):
        if opt[0]:
            convert.cmd_list.append(opt[1])
            convert.cmd_list.append("y")

    # Output filename
    convert.cmd_list.append("-f")
    convert.cmd_list.append(f"{basename}")

    # Image file   
    convert.cmd_list.append(f"{file}")

    # Create TmpDir object
    with TmpDir(tmp_dir=out_dir,use_cwd=False) as tmp_dir:
        # Create temporary output directory
        tmp_dir.mk_tmp_dir()
        
        # Output directory
        out_dir: str = os.path.abspath(out_dir)
        
        convert.cmd_list.append("-o")
        convert.cmd_list.append(f"{tmp_dir.tmp_dir}")
        
        # Execute command (assumes dcm2niix is added to system path variable)
        convert.run(log=log,env=env,dryrun=dryrun)
        
        # Copy files to output directory
        img_data = BIDSimg(work_dir=tmp_dir.tmp_dir)
        [imgs, jsons, bvals, bvecs] = img_data.copy_img_data(target_dir=out_dir)
        
        # Clean-up
        tmp_dir.rm_tmp_dir(rm_parent=False)
    
    # Image file check
    if len(imgs) == 0:
        raise ConversionError("Image conversion error. No output images were found.")

    # return imgs, jsons, bvals, bvecs
    return img_data

In [4]:
bool_opts: List[Union[str,bool]] = ["bids", "anon_bids", "gzip", "comment", "adjacent", "nrrd", "ignore_2D", "merge_2D", "text", "verbose", "lossless", "progress", "xml"]
bool_vars: List[str] = ["-b", "-ba", "-z", "-c", "-a", "-e", "-i", "-m", "-t", "-v", "-l", "--progress", "--xml"]

In [20]:
for idx,var in enumerate(bool_opts):
    if var:
        # convert.cmd_list.append(bool_vars[idx])
        # convert.cmd_list.append("y")
        print(bool_vars[idx])

-b
-ba
-z
-c
-a
-e
-i
-m
-t
-v
-l
--progress
--xml


In [23]:
for opt in zip(bool_opts,bool_vars):
    print(f"{opt[0]}:{opt[1]}")

bids:-b
anon_bids:-ba
gzip:-z
comment:-c
adjacent:-a
nrrd:-e
ignore_2D:-i
merge_2D:-m
text:-t
verbose:-v
lossless:-l
progress:--progress
xml:--xml


In [217]:
test_dir = "tmp_dir_4541"; test_dir = os.path.abspath(test_dir)

In [118]:
# path_sep: str = "/"
search_dir = os.path.join(test_dir,"*")
f = glob.glob(search_dir); f.sort(); f

['/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file1.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file2.nii.gz',
 '/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file3.json',
 '/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file4.bval',
 '/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file5.bvec']

In [63]:
f

In [2]:
class BIDSimg():
    '''File collection and organization object intended for handling 
    Brain Imaging Data Structure (BIDS) data in the processing of being converted from source data to 
    NIFTI.
    
    Attributes:
        imgs: List of NIFTI files.
        jsons: Corresponding list of JSON sidecar(s).
        bvals: Corresponding list of bval file(s).
        bvecs: Corresponding list of bvec file(s).
    '''

    def __init__(self,
                 work_dir: str):
        '''Constructs class instance lists for NIFTI, JSON, bval, and bvec files.
        
        Arguments:
            work_dir: Input working directory that contains the image files and 
                their associated output files.
        '''
        # Working directory
        self.work_dir: str = os.path.abspath(work_dir)

        # Init empty lists
        self.imgs: List[str] = []
        self.jsons: List[str] = []
        self.bvals: List[str] = []
        self.bvecs: List[str] = []
            
        # Find, organize, and sort NIFTI files
        search_dir: str = os.path.join(self.work_dir,"*.nii*")
        imgs: List[str] = glob.glob(search_dir)
        self.imgs: List[str] = imgs
        self.imgs.sort(reverse=False)
        del search_dir, imgs

        # Find and organize associated JSON, bval & bvec files
        for img in self.imgs:
            img_file: NiiFile = NiiFile(img)
            [path, file, ext] = img_file.file_parts()

            json: str = os.path.join(path,file + ".json")
            bval: str = os.path.join(path,file + ".bval")
            bvec: str = os.path.join(path,file + ".bvec")

            # JSON
            if os.path.exists(json):
                self.jsons.append(json)
            else:
                self.jsons.append("")
            
            # bval
            if os.path.exists(bval):
                self.bvals.append(bval)
            else:
                self.bvals.append("")
            
            # bvec
            if os.path.exists(bvec):
                self.bvecs.append(bvec)
            else:
                self.bvecs.append("")

            del img_file,path,file,ext,json,bval,bvec
    
    def __repr__(self):
        '''NOTE: Returns a string represented as a dictionary of list items.'''
        return ( str({"imgs": self.imgs,
                      "jsons": self.jsons,
                      "bvals": self.bvals,
                      "bvecs": self.bvecs}) )
    
    def copy_img_data(self,
                      target_dir: str
                     ) -> Tuple[List[str],List[str],List[str],List[str]]:
        '''Copies image data and their associated files to some target directory.

        NOTE: This function resets the class attributes of the class instance with the
            returns of this function.
        
        Arguments:
            target_dir: Target directory to copy files to.
            
        Returns:
            Tuple of four lists of strings that correspond to:
                - NIFTI image files
                - Corresponding JSON file(s)
                - Corresponding bval file(s)
                - Corresponding bvec file(s)
        '''
        
        # Init new lists
        imgs: List[str] = self.imgs
        jsons: List[str] = self.jsons
        bvals: List[str] = self.bvals
        bvecs: List[str] = self.bvecs
            
        # Clear class (public) list attributes
        self.imgs: List[str] = []
        self.jsons: List[str] = []
        self.bvals: List[str] = []
        self.bvecs: List[str] = []
        
        # Copy image files
        for img in imgs:
            file = copy(img,target_dir)
            self.imgs.append(file)
            
        # Copy JSON files
        for json in jsons:
            try:
                file = copy(json,target_dir)
                self.jsons.append(file)
            except FileNotFoundError:
                self.jsons.append("")
                
            
        # Copy bval files
        for bval in bvals:
            try:
                file = copy(bval,target_dir)
                self.bvals.append(file)
            except FileNotFoundError:
                self.bvals.append("")
        
        # Copy bvec files
        for bvec in bvecs:
            try:
                file = copy(bvec,target_dir)
                self.bvecs.append(file)
            except FileNotFoundError:
                self.bvecs.append("")
        
        return self.imgs, self.jsons, self.bvals, self.bvecs

In [6]:
os.getcwd()

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin'

In [3]:
f = BIDSimg("tmp_dir_4541")

In [4]:
f.imgs

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file1.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file2.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file3.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file3a.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file4.nii.gz']

In [5]:
f.jsons

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file1.json',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file2.json',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file3.json',
 '',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file4.json']

In [6]:
f.bvals

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file1.bval',
 '',
 '',
 '',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file4.bval']

In [7]:
f.bvecs

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file1.bvec',
 '',
 '',
 '',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file4.bvec']

In [8]:
f

{'imgs': ['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file1.nii.gz', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file2.nii.gz', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file3.nii.gz', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file3a.nii.gz', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file4.nii.gz'], 'jsons': ['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file1.json', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file2.json', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file3.json', '', 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541\\file4.json'], 'bvals': ['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_sou

In [12]:
test_dir_2 = "test.cp_dir"; test_dir_2 = os.path.abspath(test_dir_2); test_dir_2

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir'

In [13]:
f.copy_img_data(test_dir_2)

(['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file1.nii.gz',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file2.nii.gz',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file3.nii.gz',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file3a.nii.gz',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file4.nii.gz'],
 ['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file1.json',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file2.json',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file3.json',
  '',
  'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file4.json'],
 ['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp

In [16]:
f.imgs

['c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file1.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file2.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file3.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file3a.nii.gz',
 'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\test.cp_dir\\file4.nii.gz']

In [144]:
ff = NiiFile(f.imgs[0])

In [148]:
[h, t ,e] = ff.file_parts()

In [161]:
os.path.join(h,t + ".json")

'/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file1.json'

In [73]:
imgs: List[str] = []
jsons: List[str] = []
bvals: List[str] = []
bvecs: List[str] = []

In [74]:
search_terms: List[str] = [ ".nii", ".json", ".bval", ".bvec" ]
search_lists: List[List] = [ imgs, jsons, bvals, bvecs ]

In [80]:
for idx in range(0,len(search_lists),1):
    search_dir = os.path.join(test_dir,f"*{search_terms[idx]}*")
    search_lists[idx] = glob.glob(search_dir)

In [81]:
search_lists

[['/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file2.nii.gz',
  '/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file1.nii.gz'],
 ['/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file3.json'],
 ['/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file4.bval'],
 ['/Users/adebayobraimah/Desktop/projects/convert_source/bin/tmp_dir_4541/file5.bvec']]

In [180]:
# class BIDSimg():
#     '''class doc-string'''
    
#     def __init__(self,
#                  work_dir: str,
#                  imgs: List["str"],
#                  jsons: Optional[List["str"]] = None,
#                  bvals: Optional[List["str"]] = None,
#                  bvecs: Optional[List["str"]] = None):
#         '''Init doc-string'''

#         # Image file list
#         self.imgs: List[str] = imgs

#         # JSON file list
#         if jsons:
#             self.jsons: List[str] = jsons
#         else:
#             self.jsons = None

#         # B-values file list
#         if bvals:
#             self.bvals: List[str] = bvals
#         else:
#             self.bvals = None

#         # B-vectors file list
#         if bvecs:
#             self.bvecs: List[str] = bvecs
#         else:
#             self.bvecs = None


In [20]:
test_dir = "tmp_dir_4541"; test_dir = os.path.abspath(test_dir); test_dir

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin\\tmp_dir_4541'

In [21]:
os.getcwd()

'c:\\Users\\smart\\Desktop\\projects\\cs_med_data\\convert_source\\bin'

In [24]:
mod_path = "../tests/img.test.dir"

In [25]:
os.listdir(mod_path)

['001-001',
 '002-001',
 '003-001',
 '900XXT5',
 '901XXP5',
 '902XXY8',
 'C01-001',
 'C02-001',
 'C03-001',
 'P01-001',
 'P02-001',
 'P03-001']

In [36]:
test_file = "../tests/img.test.dir/001-001/DICOM/20211701/DWI_68dir/00000001.dcm"

In [39]:
os.listdir(test_file + "/..")

['00000001.dcm',
 '00000002.dcm',
 '00000003.dcm',
 '00000004.dcm',
 '00000005.dcm',
 '00000006.dcm',
 '00000007.dcm',
 '00000008.dcm',
 '00000009.dcm',
 '00000010.dcm',
 '00000011.dcm',
 '00000012.dcm',
 '00000013.dcm',
 '00000014.dcm',
 '00000015.dcm',
 '00000016.dcm',
 '00000017.dcm',
 '00000018.dcm',
 '00000019.dcm',
 '00000020.dcm',
 '00000021.dcm',
 '00000022.dcm',
 '00000023.dcm',
 '00000024.dcm',
 '00000025.dcm']

In [56]:
convert_image_data(test_file,"test.1",os.getcwd(),big_endian="y")

['dcm2niix.exe', '-6', '-w', '2', '--big_endian', 'y', '-b', 'y', '-ba', 'y', '-z', 'y', '-c', 'y', '-i', 'y', '-m', 'y', '-f', 'test.1', '../tests/img.test.dir/001-001/DICOM/20211701/DWI_68dir/00000001.dcm']
